# Feature Importance Using Random Forest

### Overview

Feature Selection using Random Forest algorithm

### Builds on
None

### Run time
approx. 20 minutes

### Notes


## Step 1 - Load Data

In [12]:
import os
import urllib.request

data_location = "house-sales-full.csv"
data_url = 'https://elephantscale-public.s3.amazonaws.com/data/house-prices/house-sales-full.csv'
#data_url = 'https://raw.githubusercontent.com/elephantscale/datasets/master/house-prices/house-sales-full.csv'

if not os.path.exists (data_location):
    data_location = os.path.basename(data_location)
    if not os.path.exists(data_location):
        print("Downloading : ", data_url)
        urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)


data_location: house-sales-full.csv


In [13]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

house_prices = pd.read_csv(data_location)
house_prices.sample(10)

,DocumentID,Date,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
16722,16723,4/25/11,568000,5649300580,Single Family,4/1/11,318700,0.73,"775,631.00",1,...,2.50,4,8,1982,0,2,294000,388000,98052,False
18667,18668,11/10/06,445000,6840700036,Single Family,11/1/06,421200,0.97,"459,791.00",1,...,1.00,2,7,1918,0,0,217000,174000,98122,False
21986,21987,10/30/07,327450,7883605695,Single Family,10/1/07,431300,0.99,"330,411.00",1,...,1.50,3,7,1923,0,0,187000,132000,98108,False
3301,3302,2/8/06,354950,1240100053,Single Family,2/1/06,387600,0.89,"398,540.00",1,...,1.75,3,7,1968,0,0,325000,56000,98074,False
19036,19037,1/29/07,275000,7135300065,Single Family,1/1/07,425600,0.98,"281,203.00",1,...,1.00,3,6,1918,0,0,110000,75000,98118,False
11586,11587,4/20/07,390000,3500100176,Single Family,4/1/07,430900,0.99,"393,892.00",1,...,1.00,2,6,1947,0,0,177000,67000,98155,False
23858,23859,4/1/08,615000,8682290830,Single Family,4/1/08,411700,0.95,"650,104.00",1,...,2.00,2,8,2006,0,0,245000,355000,98053,False
21989,21990,5/22/08,230000,7883608521,Single Family,5/1/08,407400,0.94,"245,695.00",1,...,1.00,3,6,1912,0,0,96000,80000,98108,True
7909,7910,2/18/10,527000,2553300120,Single Family,2/1/10,350900,0.81,"653,606.00",1,...,2.00,3,10,1994,0,0,270000,303000,98075,False
22258,22259,2/15/13,176500,7956300080,Single Family,2/1/13,347000,0.80,"221,363.00",1,...,1.75,4,6,1962,0,0,55000,141000,98023,False


## Step 2 - Explore Data (EDA)
EDA is a great way to get a sense of the data.  

Try to find answers to the following questions, by looking at the output of `describe` below

- How many 'max' bedrooms do we have? :-)
- Find min/max of 'SalePrice'
- Do you think we have outliers in data

In [14]:
house_prices.describe()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode
count,"27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00"
mean,"13,532.00","511,626.20","4,680,324,882.08","390,750.58",0.90,"570,918.76",1.02,"10,997.68","2,122.96",275.30,2.26,3.38,7.78,"1,977.09",86.73,0.21,"213,820.07","317,211.67","82,223.04"
std,"7,812.56","342,821.17","2,896,350,979.15","37,024.46",0.09,"380,236.63",0.15,"28,110.66",939.84,428.71,0.77,0.90,1.18,30.92,407.32,0.55,"177,213.41","234,038.34","36,106.67"
min,1.00,"3,000.00","1,000,102.00","311,600.00",0.72,"3,368.00",1.00,494.00,370.00,0.00,0.00,0.00,3.00,"1,900.00",0.00,0.00,0.00,0.00,-1.00
25%,"6,766.50","329,000.00","2,213,000,057.50","357,100.00",0.82,"366,918.50",1.00,"4,257.50","1,440.00",0.00,1.75,3.00,7.00,"1,954.00",0.00,0.00,"105,000.00","183,000.00","98,019.00"
50%,"13,532.00","425,000.00","3,972,900,140.00","400,600.00",0.92,"475,664.00",1.00,"6,636.00","1,940.00",0.00,2.50,3.00,8.00,"1,986.00",0.00,0.00,"172,000.00","261,000.00","98,053.00"
75%,"20,297.50","590,000.00","7,504,001,385.00","421,200.00",0.97,"655,061.00",1.00,"9,450.00","2,610.00",510.00,2.50,4.00,8.00,"2,006.00",0.00,0.00,"258,000.00","380,000.00","98,115.00"
max,"27,063.00","11,000,000.00","9,906,000,035.00","435,200.00",1.00,"11,644,855.00",5.00,"1,024,068.00","10,740.00","3,500.00",8.00,33.00,13.00,"2,016.00","2,016.00",3.00,"5,538,000.00","5,772,000.00","98,354.00"


In [15]:
house_prices.corr()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
DocumentID,1.00,-0.02,0.99,-0.02,-0.02,-0.02,-0.02,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.01,-0.01,0.05
SalePrice,-0.02,1.00,-0.02,0.12,0.12,0.98,0.02,0.14,0.68,0.29,0.52,0.32,0.66,0.08,0.09,-0.01,0.79,0.81,-0.02,0.04
PropertyID,0.99,-0.02,1.00,-0.02,-0.02,-0.02,-0.01,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.02,-0.01,0.05
zhvi_px,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
zhvi_idx,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
AdjSalePrice,-0.02,0.98,-0.02,-0.03,-0.03,1.00,0.02,0.14,0.69,0.29,0.52,0.32,0.67,0.08,0.09,-0.02,0.80,0.82,-0.03,0.04
NbrLivingUnits,-0.02,0.02,-0.01,0.02,0.02,0.02,1.00,-0.00,0.05,0.06,0.10,0.16,-0.05,-0.11,0.01,0.08,0.03,-0.00,0.04,-0.05
SqFtLot,-0.11,0.14,-0.11,-0.00,-0.00,0.14,-0.00,1.00,0.21,0.05,0.10,0.08,0.14,0.03,0.01,-0.01,0.08,0.11,0.06,-0.09
SqFtTotLiving,-0.02,0.68,-0.02,-0.01,-0.01,0.69,0.05,0.21,1.00,0.39,0.75,0.62,0.76,0.30,0.05,-0.07,0.47,0.76,-0.10,0.11
SqFtFinBasement,-0.03,0.29,-0.03,0.02,0.02,0.29,0.06,0.05,0.39,1.00,0.26,0.30,0.14,-0.22,0.11,0.05,0.29,0.28,0.10,-0.14


## Step 3 - Shape Data
Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.

**=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"**

In [16]:
input_columns= ['zhvi_px', 'zhvi_idx', 'NbrLivingUnits', 'SqFtLot', 'SqFtTotLiving', 'SqFtFinBasement', 'Bathrooms', 
                'Bedrooms', 'BldgGrade',   'YrBuilt', 'YrRenovated', 'TrafficNoise', 'LandVal', 'ImpsVal', 'ZipCode', 'NewConstruction']

label_column = ['SalePrice']

house_prices2 = house_prices[input_columns + label_column]
house_prices2

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice
0,405100,0.93,2,9373,2400,0,3.00,6,7,1991,0,0,70000,229000,98002,False,280000
1,404400,0.93,1,20156,3764,1452,3.75,4,10,2005,0,0,203000,590000,98166,True,1000000
2,425600,0.98,1,26036,2060,900,1.75,4,8,1947,0,0,183000,275000,98166,False,745000
3,418400,0.96,1,8618,3200,1640,3.75,5,7,1966,0,0,104000,229000,98168,False,425000
4,351600,0.81,1,8620,1720,0,1.75,4,7,1948,0,0,104000,205000,98168,False,240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27058,332400,0.76,1,1161,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,False,374000
27059,339400,0.78,1,1005,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,True,374000
27060,325300,0.75,1,11170,1070,0,1.00,4,6,1971,0,0,92000,130000,98055,False,165000
27061,400600,0.92,1,6223,1345,0,2.00,3,7,1939,0,0,103000,212000,98166,False,315000


In [17]:
x = house_prices2[input_columns]
y = house_prices2[label_column]

print ("x.shape = ", x.shape)
print ("y.shape = ", y.shape)

x.shape =  (27063, 16)
y.shape =  (27063, 1)


## Step 4 : Run Regression

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import  LassoLarsIC
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# algo + data --> model
# algo =  LinearRegression ()
#algo = LassoLarsIC(criterion='aic', max_iter=500)
algo = RandomForestRegressor()

model = algo.fit(x, np.ravel(y))  # <--- training
model



RandomForestRegressor()

## Step 5 : Predict

In [19]:
## TODO Predict on 'x'
predictions = model.predict(x)
predictions

array([301130.93, 912380.57, 640837.  , ..., 168842.49, 311233.5 ,
       493574.29])

In [20]:
pd.options.display.float_format = '{:,.2f}'.format

a = house_prices2[input_columns  + label_column]
a['predicted_price'] = predictions
a.sample(10)

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice,predicted_price
9502,350600,0.81,1,3819,1850,400,2.00,3,7,1924,0,0,232000,352000,98103,False,565000,"536,952.52"
15119,351200,0.81,1,4800,1470,0,1.75,2,8,1924,0,0,242000,436000,98112,False,565000,"553,684.78"
15815,400600,0.92,1,4000,1200,0,1.50,4,7,1970,0,0,110000,225000,98108,False,300000,"305,195.50"
18162,387600,0.89,1,10200,1300,0,1.50,3,7,1958,0,0,295000,82000,98007,False,359950,"360,589.38"
4697,434600,1.00,1,10658,1290,0,2.00,3,7,1953,0,2,290000,188000,98006,False,470000,"473,504.49"
2685,312300,0.72,1,12793,4590,1000,3.25,5,11,1928,0,1,831000,1448000,98199,False,1630000,"1,840,692.85"
17711,392100,0.90,1,2198,1560,380,3.50,3,8,2006,0,0,220000,311000,98116,True,550000,"540,309.39"
13042,421200,0.97,1,16526,4230,0,3.50,5,10,2008,0,0,676000,983000,-1,False,900000,"854,870.00"
22606,425600,0.98,1,10274,2770,0,2.50,4,9,1989,0,0,259000,389000,98029,False,610000,"627,510.54"
14976,411100,0.94,1,6240,1130,0,1.00,3,7,1953,0,0,190000,142000,98125,False,332000,"338,052.60"


## Step 6 : Evaluate

**Q==> Are any coefficients close to zero?  What does that mean?**

**Q==> Also inspect R2 value.  Is it decent?**

In [21]:
## print each feature and it's coefficients
coef = pd.DataFrame({"input_column" : input_columns,  
                     "feature_importance": model.feature_importances_})
coef.sort_values('feature_importance', ascending=False)

# will add upto 1.0 (100%)

,input_column,feature_importance
13,ImpsVal,0.56
12,LandVal,0.29
9,YrBuilt,0.03
4,SqFtTotLiving,0.03
1,zhvi_idx,0.02
0,zhvi_px,0.02
3,SqFtLot,0.02
8,BldgGrade,0.01
14,ZipCode,0.01
6,Bathrooms,0.01


In [22]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

## TODO : find R2 by comparing 'y' and 'predictions'
print ("R2 : " , r2_score(y, predictions))

print ("MSE : ", mean_squared_error(y, predictions))
print ("RMSE : ", sqrt(mean_squared_error(y, predictions)) )

R2 :  0.9835216236593369
MSE :  1936571926.4998293
RMSE :  44006.498684851416


## Discussion

With R2 approaching 1.0, it is very likely the RF algorithm is overfitting :-)

We need to use the hold-out method to split the data intro train/test sets to fix this.  More on this later.